In [1]:
import os
import sys
import pandas as pd
import numpy as np 
import torch
import random
import tarfile

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/NLP

/content/drive/MyDrive/NLP


In [11]:
# Klue dataset 다운
!wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000067/data/klue-sts-v1.1.tar.gz

--2022-05-23 04:09:22--  https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000067/data/klue-sts-v1.1.tar.gz
Resolving aistages-prod-server-public.s3.amazonaws.com (aistages-prod-server-public.s3.amazonaws.com)... 52.218.179.81
Connecting to aistages-prod-server-public.s3.amazonaws.com (aistages-prod-server-public.s3.amazonaws.com)|52.218.179.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1349881 (1.3M) [application/x-gzip]
Saving to: ‘klue-sts-v1.1.tar.gz’

klue-sts-v1.1.tar.g 100%[===================>]   1.29M  1.32MB/s    in 1.0s    

2022-05-23 04:09:23 (1.32 MB/s) - ‘klue-sts-v1.1.tar.gz’ saved [1349881/1349881]



In [9]:
tar_file = tarfile.open('/content/drive/MyDrive/NLP/klue-sts-v1.1.tar.gz')
tar_file.extractall(path='/content/drive/MyDrive/NLP')
tar_file.close()

In [58]:
df1 = pd.read_json('/content/drive/MyDrive/NLP/klue-sts-v1.1/klue-sts-v1.1_train.json')
test = pd.read_json('/content/drive/MyDrive/NLP/klue-sts-v1.1/klue-sts-v1.1_dev.json')

In [59]:
df1.head()

,guid,source,sentence1,sentence2,labels,annotations
0,klue-sts-v1_train_00000,airbnb-rtt,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,"{'label': 3.7, 'real-label': 3.714285714285714...","{'agreement': '0:0:0:2:5:0', 'annotators': ['0..."
1,klue-sts-v1_train_00001,policy-sampled,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,"{'label': 0.0, 'real-label': 0.0, 'binary-labe...","{'agreement': '5:0:0:0:0:0', 'annotators': ['1..."
2,klue-sts-v1_train_00002,paraKQC-sampled,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '4:2:0:0:0:0', 'annotators': ['1..."
3,klue-sts-v1_train_00003,policy-sampled,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.","{'label': 0.6000000000000001, 'real-label': 0....","{'agreement': '4:2:1:0:0:0', 'annotators': ['1..."
4,klue-sts-v1_train_00004,airbnb-rtt,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,"{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['1..."


In [60]:
df1.shape

(11668, 6)

In [61]:
# 중복문장 확인
df1.duplicated(['sentence1', 'sentence2']).sum()

7

In [62]:
# 결측치 확인
df1.isnull().sum()

guid           0
source         0
sentence1      0
sentence2      0
labels         0
annotations    0
dtype: int64

In [65]:
# 중복문장 제거
df1 = df1.drop_duplicates(['sentence1','sentence2'], keep='first', ignore_index=True)

In [68]:
# 3종류의 라벨링 binaly 1이 같은문장 2가 다른문장 
df1.labels[0]

{'binary-label': 1, 'label': 3.7, 'real-label': 3.714285714285714}

In [69]:
df1.labels[2]

{'binary-label': 0,
 'label': 0.30000000000000004,
 'real-label': 0.33333333333333304}

In [70]:
# 라벨 분리
labels = df1.labels.to_list()
labels = pd.DataFrame(labels)
print(len(labels))
labels

11661


,label,real-label,binary-label
0,3.7,3.714286,1
1,0.0,0.000000,0
2,0.3,0.333333,0
3,0.6,0.571429,0
4,4.7,4.714286,1
...,...,...,...
11656,4.0,4.000000,1
11657,0.0,0.000000,0
11658,3.7,3.666667,1
11659,4.7,4.714286,1


In [85]:
labels['label'].value_counts()
la1 = len(labels.loc[(labels['label'] >= 0) & (labels['real-label'] < 1.0)])
la2 = len(labels.loc[(labels['label'] >= 1.0) & (labels['real-label'] < 2.0)])
la3 = len(labels.loc[(labels['label'] >= 2.0) & (labels['real-label'] < 3.0)])
la4 = len(labels.loc[(labels['label'] >= 3.0) & (labels['real-label'] < 4.0)])
la5 = len(labels.loc[(labels['label'] >= 4.0) & (labels['real-label'] < 5.0)])

print(f' Score 1미만인 데이터: {la1}\n Score 2미만인 데이터: {la2}\n Score 3미만인 데이터: {la3}\n Score 4미만인 데이터: {la4}\n Score 5미만인 데이터: {la5}')  

 Score 1미만인 데이터: 4350
 Score 2미만인 데이터: 906
 Score 3미만인 데이터: 810
 Score 4미만인 데이터: 2852
 Score 5미만인 데이터: 2698


In [73]:
# binary-labele 은 score 3미만인 데이터로 이루어져 있다.
labels['binary-label'].value_counts()

0    6066
1    5595
Name: binary-label, dtype: int64

In [96]:
df1_1 = df1[['sentence1', 'sentence2']].join(labels['binary-label'])
df1_1.head()

,sentence1,sentence2,binary-label
0,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,1
1,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,0
2,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0
3,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.",0
4,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,1


In [19]:
# KorNLUdataset 다운
!git clone https://github.com/kakaobrain/KorNLUDatasets/

Cloning into 'KorNLUDatasets'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 1), reused 16 (delta 1), pack-reused 0
Unpacking objects: 100% (16/16), done.
Checking out files: 100% (9/9), done.


In [5]:
# Question pair 다운
!git clone https://github.com/songys/Question_pair

Cloning into 'Question_pair'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.


In [17]:
path = '/content/drive/MyDrive/NLP/KorNLUDatasets/KorSTS/sts-train.tsv'

In [155]:
df2 = pd.read_csv(path, sep="\t+")

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [156]:
# 스코어로 되어있는 형태
df2

,genre,filename,year,id,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,1,5.00,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.
1,main-captions,MSRvid,2012test,4,3.80,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.
2,main-captions,MSRvid,2012test,5,3.80,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.
3,main-captions,MSRvid,2012test,6,2.60,세 남자가 체스를 하고 있다.,두 남자가 체스를 하고 있다.
4,main-captions,MSRvid,2012test,9,4.25,한 남자가 첼로를 연주하고 있다.,자리에 앉은 남자가 첼로를 연주하고 있다.
...,...,...,...,...,...,...,...
5744,main-news,headlines,2016,1456,0.00,폭풍우 클로다흐가 영국을 강타하면서 심한 강풍,메르켈은 나토와 라트비아의 연대를 약속한다.
5745,main-news,headlines,2016,1465,0.00,리비아 테러리스트들이 공습에 대한 복수로 찍은 수십 명의 이집트 인질들,나일강에서 더 많은 시체가 발견되면서 이집트 보트 충돌 사고 사망자 수가 증가한다.
5746,main-news,headlines,2016,1466,0.00,바레인으로 향하는 대통령,시 주석 : 에볼라 퇴치를 계속 돕기 위한 중국
5747,main-news,headlines,2016,1470,0.00,"중국, 인도는 양국 관계를 증진시키겠다고 맹세한다",중국은 불안한 주식 거래자들을 안심시키기 위해 뒤뚱거리고 있다.


In [157]:
# 중복값 확인
df2.duplicated(['sentence1', 'sentence2']).sum()

49

In [158]:
# 중복값 제거
df2 = df2.drop_duplicates(['sentence1','sentence2'], keep='first', ignore_index=True)

In [159]:
# 결측치 확인
df2.isnull().sum()

genre        0
filename     0
year         0
id           0
score        0
sentence1    0
sentence2    0
dtype: int64

In [160]:
# 위와 같이 스코어 3 미만은 0 3 이상은 1 로 분류
df2['binary-label'] = df2['score'].apply(lambda x: 0 if x < 3  else  1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [161]:
# 스코어 확인
la11 = len(df2.loc[(df2['score'] >= 0) & (df2['score'] < 3.0)])
la11

2729

In [162]:
# 분류 확인
df2['binary-label'].value_counts()

1    2971
0    2729
Name: binary-label, dtype: int64

In [163]:
df2_1 = df2[['sentence1', 'sentence2', 'binary-label']]
df2_1.head()

,sentence1,sentence2,binary-label
0,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.,1
1,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.,1
2,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.,1
3,세 남자가 체스를 하고 있다.,두 남자가 체스를 하고 있다.,0
4,한 남자가 첼로를 연주하고 있다.,자리에 앉은 남자가 첼로를 연주하고 있다.,1


In [164]:
path2 = '/content/drive/MyDrive/NLP/Question_pair/kor_pair_train.csv'

In [165]:
df3 = pd.read_csv(path2)

In [166]:
# 1이 다른문장 2가 같은문장
df3

,id,qid1,qid2,question1,question2,is_duplicate
0,1,1,2,1000일 만난 여자친구와 이별,10년 연예의끝,1
1,2,3,4,10년 연애사 되돌아보니 다 부질없네,10년이라는 시간이 참 무색하다 싶네,1
2,3,5,6,10년만나다 헤어지네,14년된 여자친구랑 이별,1
3,4,7,8,10월의 마지막밤 . 더 보고싶네 그사람,15년…안녕,1
4,5,9,10,14년의 기나긴 이야기.,1년 9개월 의 연애 종지부.,1
...,...,...,...,...,...,...
6883,6884,13767,13768,O형 남자와 썸을 타는데,O형 여자와 썸을 타는데,1
6884,6885,13769,13770,SD카드 안돼,SD카드 망가졌어,0
6885,6886,13771,13772,SNS 를 끊어야 하는데.,가슴 아픈 이별,1
6886,6887,13773,13774,SNS 시간낭비인데 자꾸 보게됨,SNS 시간낭비인 거 아는데 매일 하는 중,0


In [167]:
# 중복값 확인
df3.duplicated(['question1', 'question2']).sum()

1

In [168]:
# 중복값 제거
df3 = df3.drop_duplicates(['question1', 'question2'], keep='first', ignore_index=True)

In [169]:
# 결측치 확인
df3.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [170]:
# 분류 기준 바꾸기
df3['binary-label'] = df3['is_duplicate'].apply(lambda x: 0 if x > 0 else 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [171]:
# 분류 확인
df3['is_duplicate'].value_counts()

0    4749
1    2138
Name: is_duplicate, dtype: int64

In [172]:
# 분류 확인
df3['binary-label'].value_counts()

1    4749
0    2138
Name: binary-label, dtype: int64

In [173]:
df3_1 = df3[['question1', 'question2', 'binary-label']]

In [174]:
df3_1.rename(columns = {'question1':'sentence1','question2':'sentence2'},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [181]:
df = pd.concat([df1_1,df2_1, df3_1],axis=0, join='inner', ignore_index=True) 

In [182]:
df

,sentence1,sentence2,binary-label
0,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,1
1,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,0
2,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0
3,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.",0
4,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,1
...,...,...,...
24243,O형 남자와 썸을 타는데,O형 여자와 썸을 타는데,0
24244,SD카드 안돼,SD카드 망가졌어,1
24245,SNS 를 끊어야 하는데.,가슴 아픈 이별,0
24246,SNS 시간낭비인데 자꾸 보게됨,SNS 시간낭비인 거 아는데 매일 하는 중,1
